In [1]:
import tensorflow as tf
import numpy as np

In [16]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        // tf-graph-basic.build.html?download
        // http://localhost:8080/tf-graph-basic.build.html
        // https://tensorboard.appspot.com/tf-graph-basic.build.html
        </script>
        <link rel="import" href="http://localhost:8888/files/tf-graph/tf-graph-basic.build.html" onload=load()>
        
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [35]:
tf.reset_default_graph()
a = tf.constant(1.2)
b = tf.constant([3.14, 7, -1])
with tf.Session().as_default():
    print((a + b * a).eval())

[ 4.96800041  9.60000038  0.        ]


In [37]:
show_graph(tf.get_default_graph())

In [5]:
g=tf.get_default_graph()

In [6]:
[op.name for op in g.get_operations()]

['Const', 'Const_1', 'mul', 'add']

In [39]:
tf.reset_default_graph()
batch_size = 100
x = tf.placeholder(tf.float32, [batch_size, 784], name='image')
W = tf.Variable(tf.zeros([784, 10]), name='weights')
b = tf.Variable(tf.zeros([10]), name='bias')
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.int32, [batch_size], name='expected')

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=y)
mean = tf.reduce_mean(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(mean)
show_graph(tf.get_default_graph())

In [89]:
[(op.name, op.type) for op in tf.get_default_graph().get_operations()]

[('image', 'Placeholder'),
 ('zeros', 'Const'),
 ('weights', 'Variable'),
 ('weights/Assign', 'Assign'),
 ('weights/read', 'Identity'),
 ('zeros_1', 'Const'),
 ('bias', 'Variable'),
 ('bias/Assign', 'Assign'),
 ('bias/read', 'Identity'),
 ('MatMul', 'MatMul'),
 ('add', 'Add'),
 ('expected', 'Placeholder'),
 ('SparseSoftmaxCrossEntropyWithLogits/Shape', 'Const'),
 ('SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits',
  'SparseSoftmaxCrossEntropyWithLogits'),
 ('Const', 'Const'),
 ('Mean', 'Mean'),
 ('gradients/Shape', 'Const'),
 ('gradients/Const', 'Const'),
 ('gradients/Fill', 'Fill'),
 ('gradients/Mean_grad/Reshape/shape', 'Const'),
 ('gradients/Mean_grad/Reshape', 'Reshape'),
 ('gradients/Mean_grad/Tile/multiples', 'Const'),
 ('gradients/Mean_grad/Tile', 'Tile'),
 ('gradients/Mean_grad/Shape', 'Const'),
 ('gradients/Mean_grad/Shape_1', 'Const'),
 ('gradients/Mean_grad/Const', 'Const'),
 ('gradients/Mean_grad/Prod', 'Prod'),
 ('gradients/Mean_grad/Const_1', 'Cons

In [33]:
tf.reset_default_graph()
batch_size = None
x = tf.placeholder(tf.float32, [batch_size, 784], name='image')
W = tf.Variable(tf.zeros([784, 10]), name='weights')
b = tf.Variable(tf.zeros([10]), name='bias')
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.int32, [batch_size, 10], name='expected')

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
mean = tf.reduce_mean(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(mean)
show_graph(tf.get_default_graph())